## Concept Drift in Regression Data

In [ ]:
# import the functions needed for validate and comparsion

import numpy as np
import cupy as cp
import pandas as pd
from timeit import default_timer as timer
import os

from sklearn.model_selection import train_test_split
#mean_squared_error: mse squared=true, rmse squared=false
from sklearn.metrics import mean_squared_error
from skmultiflow.data import ConceptDriftStream, FileStream
from sklearn.linear_model import SGDRegressor
from skmultiflow.data.regression_generator import RegressionGenerator
from sklearn.preprocessing import MinMaxScaler


# using plotly for plots
import plotly.express as px
from plotly.subplots import make_subplots


In [ ]:
from fires import FIRES

In [ ]:
from stability import stability_factor

In [ ]:
stream = ConceptDriftStream(stream=RegressionGenerator(random_state=42, n_features=100, n_informative=25), drift_stream=RegressionGenerator(random_state=52, n_features=100, n_informative=25), position=10100, width=10)

dataset_name = "Concept_drift_10"
n_selected_ftr = 25
n_window = 100
batch_size = 10

In [ ]:

stream = ConceptDriftStream(stream=RegressionGenerator(random_state=42, n_features=100, n_informative=25), drift_stream=RegressionGenerator(random_state=52, n_features=100, n_informative=25), position=10100, width=1000)

dataset_name = "Concept_drift_1000"
n_selected_ftr = 25
n_window = 100
batch_size = 10

In [ ]:
# prepare folder for plots
folder = "plots/regression/{}".format(dataset_name)
if not os.path.exists(folder):
    os.makedirs(folder)

export_type = "pdf" # "png", "jpeg", "webp", "pdf", "svg"

In [ ]:
# use SGDRegressor as predictor
stream.restart()
predictor = SGDRegressor()
scaler = MinMaxScaler()
scaler_y = MinMaxScaler()
X, y = stream.next_sample(batch_size=100)
scaler.partial_fit(X)
scaler_y.partial_fit(y.reshape(-1,1))

X = scaler.transform(X)
y = scaler_y.transform(y.reshape(-1,1))
predictor.partial_fit(X,y)

In [ ]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=None,
                    mu_init=0,
                    sigma_init=1,
                    model='regression')  

In [ ]:

fires_rmse = []


fires_selected_ftrs = []
fires_stability = []

start_time_all = timer()
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=10)
    scaler.partial_fit(x)
    scaler_y.partial_fit(y.reshape(-1,1))
    x = scaler.transform(x)
    y = scaler_y.transform(y.reshape(-1,1))
    # Select features
    ftr_weights = fires_model.weigh_features(x,y)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fires_selected_ftrs.append(ftr_array)

    if len(fires_selected_ftrs) >= 10:
        stability = stability_factor(fires_selected_ftrs[-10:])
        fires_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    fires_rmse.append(mean_squared_error(y, y_pred, squared=False))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
fires_rmse = pd.Series(fires_rmse).rolling(window=n_window).mean().iloc[n_window-1:].values
end_time_all = timer()
fires_run_time = timer() - start_time_all
print("The whole fires run took {}".format(fires_run_time))

In [ ]:
# use SGDRegressor as predictor
stream.restart()
predictor = SGDRegressor()
scaler = MinMaxScaler()
scaler_y = MinMaxScaler()
X, y = stream.next_sample(batch_size=100)
scaler.partial_fit(X)
scaler_y.partial_fit(y.reshape(-1,1))

X = scaler.transform(X)
y = scaler_y.transform(y.reshape(-1,1))
predictor.partial_fit(X,y)

In [ ]:
sgdr_model = SGDRegressor(penalty="l1") #penalty could be elasticnet as well
#n_selectey_ftrs?

In [ ]:
sgdr_rmse = []

sgdr_selected_ftrs = []
sgdr_stability = []

start_time_all = timer()
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=10)
    scaler.partial_fit(x)
    scaler_y.partial_fit(y.reshape(-1,1))
    x = scaler.transform(x)
    y = scaler_y.transform(y.reshape(-1,1))
    # Select features
    sgdr_model.partial_fit(x,y)
    ftr_weights = sgdr_model.coef_
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    sgdr_selected_ftrs.append(ftr_array)

    if len(sgdr_selected_ftrs) >= 10:
        stability = stability_factor(sgdr_selected_ftrs[-10:])
        sgdr_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    sgdr_rmse.append(mean_squared_error(y, y_pred, squared=False))
    
    


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
sgdr_rmse = pd.Series(sgdr_rmse).rolling(window=n_window).mean().iloc[n_window-1:].values
end_time_all = timer()
sgdr_run_time = timer() - start_time_all
print("The whole sgdr run took {}".format(sgdr_run_time))


In [ ]:
# stability
title = "Stability on dataset {}".format(dataset_name)
col_names = ["FIRES", "SGDR"]
d = {"FIRES":fires_stability, "SGDR":sgdr_stability}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y = col_names, title=title, labels={"index":"batches", "value":"stability"}, color_discrete_map={'FIRES': 'red', "SGDR": "green"})
stability_trace = fig["data"]
fig.show()

In [ ]:
title = "RMSE on dataset {}".format(dataset_name)
col_names = ["FIRES", "SGDR"]
d = {"FIRES":fires_rmse, "SGDR":sgdr_rmse}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y = col_names, title=title, labels={"index":"batches", "value":"rmse"}, color_discrete_map={'FIRES': 'red', "SGDR": "green"})
rmse_trace = fig["data"]
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1) # subplot_titles=("Stability","Accuracy", "F1-Score"),
for i in range(len(stability_trace)):
    stability_trace[i]["showlegend"] = False
    if stability_trace[i]["name"] == "FIRES":
        trace = stability_trace[i]
    else:
        fig.add_trace(stability_trace[i], row=1, col=1)
fig.add_trace(trace, row=1, col=1)
for i in range(len(rmse_trace)):
    fig.add_trace(rmse_trace[i], row=2, col=1)
fig.update_xaxes(title_text="batches", row=2, col=1)
fig.update_yaxes(title_text="Stability", row=1, col=1)
fig.update_yaxes(title_text="RMSE", row=2, col=1)

fig.write_image("{}/{}.{}".format(folder, dataset_name, export_type))
fig.show()